# Configuring the Slow Control on the Hawaii Test bench


## Drawing of the Hawaii setup

![setup](img/BII-KLM Testbench HW Diagram.png)


## preparing the script
First we have to load some auxillary libraries and configurations for running the script

In [ ]:
import time
import  config.config_hawaii as  config
from Linux_Helpers.remoteshell2 import remoteShell2
from Linux_Helpers.py_reghs import py_reghs
from equipment.ScintilatorReadout import ScintilatorReadout
import Input_checks
import config.registers_DC as  registers_DC
from equipment.py_hslb import  py_hslb
from equipment.py_ftsw import  py_ftsw
from config.local_run_ROI import roi,visualise_ROIs

from Jupyter_helpers.Controlflow import button_run_below


Next we have to connect to the `vme controller` and the `copper`. 

In [ ]:
vme = remoteShell2(config.vme_tunnel)
ftsw = py_ftsw(vme,config.FTSW_HI)

In [ ]:
cpr = remoteShell2(config.copper_tunnel)
hslb =py_hslb(cpr,config.hslb_cpr107)

## Checking if copper / vme are configured correctly

The lines above should be executed without any complaint. 

In [ ]:
print(hslb.status())

the output should look something like this:
<details><summary> `expected`</summary>
<p>

```
staths version 5 (20160114)
HSLB-b version 0.61 / KLM firmware 1 serial 0 version 4
(b)    stat=18001002 (ff=0 rx=1 pr=0 pt=0 tx=18 masked)
(b)    rxdata=00bc rxlinkdown=0 rxcrcerr=0 feecrcerr=0
(b)    event=2153958 total=86158kB (avg=40B last=40B max=40B)
(b)    no b2link error
```

</p>
</details>

if it looks any different boot the HSLB using:


In [ ]:
print(hslb.boot())

after this make sure the `vme` controller can communicate with the `FTSW`. You can check this with the following command:

In [ ]:
print(ftsw.status())

<details><summary> `expected` </summary>
<p>
the output should be something like this:
</p>
<p>

```
statft version 20160426 FTSW #081 / ft2u090j - 2018.09.12 21:27:07.620
warning: time is not yet set, do either bootft or utimeft
```
</p>
<p>
if the output looks like this:
</p>
<p>
```
statft: FTSW#080 not found: id=ffffffff
```
</p>
<p>
then the id for the FTSW is wrong

</p>
</details>
    

# Initiliizing the copper readout 
After this the Scintillator readout needs to be initilized using the configuration provided in the `newScripts` folder

In [ ]:
SCINT_PKT_SZ = registers_DC.Run_Control.SCINT_PKT_SZ
hslb.reghs(SCINT_PKT_SZ)
hslb.reghs_setAndCheck(SCINT_PKT_SZ ,config.SlowControl.first_reg_value)
hslb.reghs_setAndCheck(SCINT_PKT_SZ ,config.SlowControl.second_reg_value)
ret = hslb.reghs_stream_file(config.SlowControl.initFile)

This should run without any feedback. If it cannot configure the SCINT_PKT_SZ it will throw an exception. There is no way of checking if the init (dat) file was processed correctly.

## Starting a Run

After the setup is configured/initilized the run can be started by first resetting the `FTSW` with the `resetft -81` command and then using the `trigft -81  pulse 1000 -1` command to start the issuing of triggers. Both from the `vme` controller

In [ ]:
button=button_run_below("Starting a Run")

In [ ]:
print(ftsw.reset())
time.sleep(1)

<details><summary> `expected`</summary>
<p>

```
trigft version 2016062700
resetting trigger

```

</p>
</details>

In [ ]:
print(ftsw.trigger(rate=1000))

<details><summary> `expected`</summary>
<p>

```
trigft version 2016062700
pulse trigger rate 1000.590 Hz
exp 0 run 12 sub 0 started

```

</p>
</details>

In [ ]:
visualise_ROIs(hslb,roi,3)

<details><summary> `expected`</summary>
<p>
```
FIFO_FLAGS1	 FIFO_FLAGS2	 MISSED_TRG	 SCINT_TRGTAG	 EVNT_RDCNT1	 EVNT_WRCNT1	 EVNT_WDCNT	 
	 HSL_CHANNEL_UP	 DAQ_PKTSZ1	 DAQ_PKTSZ2	 SCINT_FLAGS	 SCINT_EVTAG	 EVNT_RDCNT2	 EVNT_WRCNT2	 

255	 264	 0	     264	 0	 3	 8704	 0	 51322	 345	 531	 674	 859	 0	 
255	 264	 0	     264	 0	 3	 8704	 0	 54003	 995	 163	 276	 467	 0	 
255	 264	 0	     264	 0	 3	 8704	 0	 56599	 507	 704	 887	 46	  0	 
```

</p>
</details>

<details><summary> `error`</summary>
<p>
```
FIFO_FLAGS1	 FIFO_FLAGS2	 MISSED_TRG	 SCINT_TRGTAG	 EVNT_RDCNT1	 EVNT_WRCNT1	 EVNT_WDCNT	 
	 HSL_CHANNEL_UP	 DAQ_PKTSZ1	     DAQ_PKTSZ2	 SCINT_FLAGS	 SCINT_EVTAG	 EVNT_RDCNT2	 EVNT_WRCNT2	 
255	 264	 65535	 264	 65535	 1665	 512	 0	 31838	 2	 0	 152	 0	 0	 
255	 264	 65535	 264	 65535	 1665	 512	 0	 31838	 2	 0	 152	 0	 0	 
255	 264	 65535	 264	 65535	 1665	 512	 0	 31838	 2	 0	 152	 0	 0	 
```

</p>
</details>


If the system is in the error state try to rerun the Process starting from "Starting a Run" command.